# GOLD

**Tipo de tabla:** Hechos de lineas de detalles de ventas

**Origen:**  `orders_header` `orders_details` 

**Destino:** `fact_order_line`

## Lectura de datos parquet de Bronze

In [ ]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

load_dotenv("/home/jovyan/work/.env")

spark = SparkSession.builder.appName("gold_dim_features").config("spark.driver.memory", "4g").getOrCreate()

gold_path = os.getenv("GOLD_PATH")
silver_path = os.getenv("SILVER_PATH")

## orders_lines

## Tablas extras dimensionales para posteriores cruces

In [ ]:
# tablas dimensionales para cruce de info
customer_path=  os.path.join(silver_path, "dim_customer.parquet")
spark.read.parquet(customer_path).createOrReplaceTempView("dim_customer")

product_path= os.path.join(silver_path, "dim_product.parquet")
spark.read.parquet(product_path).createOrReplaceTempView("dim_product")

date_path= os.path.join(silver_path, "dim_date.parquet")
spark.read.parquet(date_path).createOrReplaceTempView("dim_date")

order_line_path= os.path.join(silver_path, "fact_order_line.parquet")
spark.read.parquet(order_line_path).createOrReplaceTempView("fact_order_line")

## Creacion de tabla gold para team
### Reglas de Limpieza aplicada:
* Tipos Correctos
* Eliminacion de Duplicados por customer_id
* Evitar * en selects

### Reglas de Transformacion:
* Se agrega dato dummy para consistencia en productos no existentes
* Estandarizacion en lenguaje de nombres de columnas

In [4]:
query = """
with cte_customer_ped_3m AS(
    SELECT cus.customer_id
        , MAX(dat.date) AS last_pedido_date
        , last_day(current_date()) as month_end_date
        , COUNT(DISTINCT lin.order_id) as frecuencia_pedidos_3m 
        , SUM(lin.line_net_amount) AS ventas_total_3m
        , COUNT(DISTINCT pro.product_category) AS variedad_categorias_3m
        , COUNT(DISTINCT CASE WHEN lin.promo_flag = 1 THEN lin.order_id END) AS n_orders_promo
        , SUM(CASE WHEN lin.pay_type = 'credito' THEN lin.line_net_amount ELSE 0 END) AS venta_credito_3m
    FROM fact_order_line AS lin
    LEFT JOIN dim_customer AS cus
        ON lin.customer_id = cus.customer_id
    LEFT JOIN dim_product AS pro
        ON lin.product_id = pro.product_id
    LEFT JOIN dim_date AS dat
        ON lin.order_date_id = dat.date_id
    WHERE cus.channel = 'tradicional'
    AND lin.period >= CAST(date_format(add_months(current_date(), -2), 'yyyyMM') AS INT)
    GROUP BY ALL
),
cte_customer_dias_promedio AS (
    SELECT *
        , LAG(order_date) OVER(PARTITION BY customer_id ORDER BY order_date, order_id) AS last_date
    FROM (
        SELECT cus.customer_id
            , lin.order_id
            , dat.date AS  order_date
        FROM fact_order_line AS lin
        LEFT JOIN dim_customer AS cus
            ON lin.customer_id = cus.customer_id
        LEFT JOIN dim_date AS dat
            ON lin.order_date_id = dat.date_id
        WHERE cus.channel = 'tradicional'
        AND lin.period >= CAST(date_format(add_months(current_date(), -2), 'yyyyMM') AS INT)
        GROUP BY ALL
    )
),
cte_avg_dias_promedio AS (
    SELECT customer_id, AVG(datediff( order_date, last_date)) AS dias_promedio_entre_pedidos
    FROM cte_customer_dias_promedio
    group by all
)
SELECT ped.customer_id
    , DATEDIFF(ped.month_end_date, ped.last_pedido_date) AS recencia_dias
    , ped.ventas_total_3m
    , ped.frecuencia_pedidos_3m
    , (ped.ventas_total_3m/ ped.frecuencia_pedidos_3m) AS ticket_promedio_3m
    , avg.dias_promedio_entre_pedidos
    , ped.variedad_categorias_3m
    , (ped.n_orders_promo/ ped.frecuencia_pedidos_3m) AS porcentaje_pedidos_promo_3m
    , (ped.venta_credito_3m / ped.ventas_total_3m) AS mix_credito_vs_contado_3m
FROM cte_customer_ped_3m AS ped
LEFT JOIN cte_avg_dias_promedio AS avg
    ON ped.customer_id = avg.customer_id
WHERE ped.frecuencia_pedidos_3m > 0


"""
# Execute the SQL query and get the result as a new DataFrame
lines_df = spark.sql(query)

# Display the results
lines_df.printSchema()
lines_df.show()

root
 |-- customer_id: integer (nullable = true)
 |-- recencia_dias: integer (nullable = true)
 |-- ventas_total_3m: decimal(28,2) (nullable = true)
 |-- frecuencia_pedidos_3m: long (nullable = false)
 |-- ticket_promedio_3m: decimal(38,12) (nullable = true)
 |-- dias_promedio_entre_pedidos: double (nullable = true)
 |-- variedad_categorias_3m: long (nullable = false)
 |-- porcentaje_pedidos_promo_3m: double (nullable = true)
 |-- mix_credito_vs_contado_3m: decimal(38,10) (nullable = true)

+-----------+-------------+---------------+---------------------+-------------------+---------------------------+----------------------+---------------------------+-------------------------+
|customer_id|recencia_dias|ventas_total_3m|frecuencia_pedidos_3m| ticket_promedio_3m|dias_promedio_entre_pedidos|variedad_categorias_3m|porcentaje_pedidos_promo_3m|mix_credito_vs_contado_3m|
+-----------+-------------+---------------+---------------------+-------------------+---------------------------+---------

## Escritura de datos
* Todo se escribe en parquet, en carpetas de silver.
* Se escribe con metodo upsert para posteriores ingestas masivas de datos
* Posibilidad de realizar SCD para preservar cambios historicos en dimensiones

**Mejora** utilziar deltas tables para control de merges y log (ACID)

In [ ]:
# Escritura de dim_product en silver como parquet
output_path = os.path.join(gold_path, "dim_features")

lines_df.write.mode("overwrite").format("parquet").save(output_path)

## Validaciones.

In [6]:
summary_stats = spark.read.parquet(output_path).describe()
summary_stats.show()

+-------+------------------+-----------------+----------------+---------------------+--------------------+---------------------------+----------------------+---------------------------+-------------------------+
|summary|       customer_id|    recencia_dias| ventas_total_3m|frecuencia_pedidos_3m|  ticket_promedio_3m|dias_promedio_entre_pedidos|variedad_categorias_3m|porcentaje_pedidos_promo_3m|mix_credito_vs_contado_3m|
+-------+------------------+-----------------+----------------+---------------------+--------------------+---------------------------+----------------------+---------------------------+-------------------------+
|  count|              8664|             8664|            8664|                 8664|                8664|                       8485|                  8664|                       8664|                     8664|
|   mean| 5449.878924284395|35.62580794090489|  2615644.867749|   16.390927977839336|159721.2837692560...|          5.671695392466507|     4.95867959372